In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import os
from cleantext import clean

In [2]:
arr = os.listdir('.')
print(arr)

['FinalDBelon.csv', 'Data_train.csv', 'DataNONEL_clean.csv', 'Clean_Tweets_EM.ipynb', 'Creation_matrice_Detection.ipynb', 'Création_matrice_Création.ipynb', 'README.md', 'Tweets_Not_ElonMusk.csv', 'DataBase.csv', '.gitignore', 'Reseau_neurone.ipynb', 'DataElon_clean.csv', 'Vectorisation.ipynb', '.ipynb_checkpoints', 'Label_train.csv', 'Clean_Tweets_NONEM.ipynb', '.git', 'TweetsElonMusk.csv']


In [3]:
DataElon = pd.read_csv('DataElon_clean.csv')
DataNONElon = pd.read_csv('DataNONEL_clean.csv')

In [4]:
def cleaner(Database_init):
    Database = Database_init.copy()   
    for i in Database.index:
        Database.loc[i, 'tweet'] = str(Database.loc[i, 'tweet']).lower()
        if '@' in str(Database.loc[i,"tweet"]):
            Tweet = Database.loc[i, 'tweet'].split(' ')
            for index, mot in enumerate(Tweet):
                if '@' in mot:
                    del Tweet[index]
            Database.loc[i, 'tweet'] = ' '.join(Tweet)
        
        Database.loc[i, 'tweet'] = clean(Database.loc[i, 'tweet'], no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
        
        
    return Database

In [5]:
def creation_mot(Liste):
    liste = [' '.join(Liste)]
    Liste_mots_doublon = liste[0].split(' ')
    for index, mot in enumerate(Liste_mots_doublon):
        if '@' in mot:
            Liste_mots_doublon[index] = ''
    Liste_mots = list(set(Liste_mots_doublon))
    return Liste_mots

In [6]:
def creation_matrice(Liste_mots, Liste):
    row =[]
    
    dictionnary = dict()
    for mot in Liste_mots:
        dictionnary[mot] = 0
    
    for tweet in Liste:
        dictionnary2 = dictionnary.copy()
        tweet_actual = tweet.split(" ")
        tweet_actual_clean = []
        for mot_tweet in tweet_actual:
            if mot_tweet != '':
                tweet_actual_clean.append(mot_tweet)
        for index, mot in enumerate(tweet_actual_clean):
            if mot in Liste_mots:
                dictionnary2[mot] = index + 1 
        row.append(dictionnary2)
    return DataFrame(row)

In [7]:
def optimize_matrice(Dataframe, n_min):
    for col in Dataframe.columns:
        if len(Dataframe[Dataframe[col] != 0]) < n_min:
            Dataframe.drop(columns = col, inplace=True)

In [8]:
DataElon = cleaner(DataElon)
Liste_tweets_EM = DataElon["tweet"].values

DataNONElon = cleaner(DataNONElon)
DataNONElon.dropna(inplace=True)
Liste_tweets_NONEM = DataNONElon["tweet"].values

TypeError: clean() got an unexpected keyword argument 'no_emoji'

In [9]:
Liste_tweets = np.concatenate((Liste_tweets_EM[:60], Liste_tweets_NONEM[:60]))

In [10]:
len(Liste_tweets)

120

In [11]:
Liste_mots = creation_mot(Liste_tweets)
len(Liste_mots)

601

In [12]:
DataBase_EM = creation_matrice(Liste_mots, Liste_tweets_EM[:60])
DataBase_EM.drop(columns = '', axis=1, inplace=True)

In [13]:
DataBase_NEM = creation_matrice(Liste_mots, Liste_tweets_NONEM[:60])
DataBase_NEM.drop(columns = '', axis=1, inplace=True)

In [14]:
DataBase = pd.concat([DataBase_EM, DataBase_NEM], ignore_index = True)

In [15]:
DataBase

,cheer,idk,veteran,many,harder,go,snowflake,protecting,precision,midnight,...,looking,back,used,mass,tribe,which,possible,stop,skyscraper,freedom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
optimize_matrice(DataBase, 3)

In [17]:
DataBase

,just,will,earth,do,a,pretty,way,beautiful,have,for,...,out,great,this,your,than,to,are,wholemarsblog,spacex,one
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,4,0,0,0
1,0,0,0,0,0,0,0,3,0,0,...,0,0,2,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117,11,0,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,0,0,0
118,0,0,0,0,0,0,0,0,7,0,...,0,0,0,4,0,0,0,0,0,0


In [19]:
Liste_tweets[0]

'for now costs are decreasing rapidly'

In [54]:
DataBase_word = pd.read_csv('FinalDBelon.csv')

In [60]:
DataBase_word

,federal,engineers,sea,going,doesnt,purposetour,america,model,health,boca,...,evafoxu,welcome,boards,join,la,guys,guess,taylorlurking,are,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11128,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
DataBase_word['label'] = (DataBase_word.index < 6000).astype(np.int64)
L =[]
for i in DataBase_word.index:
    if DataBase_word.iloc[i, :-1].sum() == 0:
        L.append(i)
DataBase_word.drop(index = L, inplace=True)

existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe
existe

In [59]:
DataBase_word.reset_index(drop=True, inplace=True)

In [64]:
DataBase_word.iloc[:,:-1]

,federal,engineers,sea,going,doesnt,purposetour,america,model,health,boca,...,countries,evafoxu,welcome,boards,join,la,guys,guess,taylorlurking,are
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11128,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
DPPP = DataBase_word.iloc[:,-1:].copy()

In [75]:
DPPP.loc[0, 'label'] = 0

In [72]:
Matrice_train = DataBase_word.iloc[:,:-1]
Label_train = DataBase_word.iloc[:,-1:]

In [97]:
def normalize(Matrix):
    Matrix_out = Matrix.copy()
    index = Matrix_out.index
    for ind in index:
        Matrix_out.loc[ind] = Matrix_out.loc[ind] / Matrix_out.loc[ind].max()
    return Matrix_out

In [98]:
DataTrain = normalize(Matrice_train)
DataTrain.to_csv("Data_train.csv", index=False)

KeyboardInterrupt: 

In [100]:
Label_train.to_csv("Label_train.csv", index=False)